In [4]:
import os
from ultralytics import YOLO
import zipfile
import cv2

# 작업 경로 설정 (로컬 경로)
base_path = "/Users/ihanjo/Documents/K-Digital/deepAI_Pytorch_my/4-Yolov8/YOLOv8"
os.makedirs(base_path, exist_ok=True)


In [5]:
# ultralytics 설치 (터미널에서 실행 필요)
# pip install ultralytics

# YOLO 설치 확인
# YOLO.checks()


In [6]:
# 데이터셋 다운로드 및 압축 해제
dataset_url = "https://public.roboflow.com/ds/EUBO4ATjbt?key=dnYrjDbMEB"
zip_path = os.path.join(base_path, "Aquarium_DataSet.zip")
extract_path = os.path.join(base_path, "Aquarium_DataSet")

# wget 대신 urllib 사용
import urllib.request
urllib.request.urlretrieve(dataset_url, zip_path)

if not os.path.exists(extract_path):
    os.makedirs(extract_path)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"Aquarium_DataSet 압축 해제가 완료되었습니다. 폴더 경로: {extract_path}")


Aquarium_DataSet 압축 해제가 완료되었습니다. 폴더 경로: /Users/ihanjo/Documents/K-Digital/deepAI_Pytorch_my/4-Yolov8/YOLOv8/Aquarium_DataSet


In [7]:
# YAML 파일 생성
yaml_path = os.path.join(extract_path, "Aquarium_Data.yaml")
data = {
    'train': os.path.join(extract_path, "train", "images") + "/",
    'val': os.path.join(extract_path, "valid", "images") + "/",
    'test': os.path.join(extract_path, "test", "images") + "/",
    'nc': 7,
    'names': ['fish', 'jellyfish', 'penguin', 'puffin', 'shark', 'starfish', 'stingray']
}

import yaml
with open(yaml_path, 'w') as f:
    yaml.dump(data, f)

print("YAML 파일 생성 완료:", yaml_path)


YAML 파일 생성 완료: /Users/ihanjo/Documents/K-Digital/deepAI_Pytorch_my/4-Yolov8/YOLOv8/Aquarium_DataSet/Aquarium_Data.yaml


In [8]:
# 사전 학습된 YOLO 모델 로드
model = YOLO("yolov8n.pt")  # 사전 학습된 모델 로드

# 모델 학습
results = model.train(
    data=yaml_path,         # 데이터셋 구성 파일 경로
    epochs=50,             # 학습 에포크 수
    patience=30,            # 조기 종료 기준 에포크 수
    batch=64,               # 배치 크기
    imgsz=416,              # 입력 이미지 크기
    project=base_path,      # 결과 저장 경로
    name='modelresult',     # 프로젝트 이름
    device="mps"            # Apple Silicon GPU(MPS) 사용
)

print(f"모델 학습 완료. 저장 경로: {results.save_dir}")


100%|██████████| 6.25M/6.25M [00:00<00:00, 8.00MB/s]


Ultralytics 8.3.95 🚀 Python-3.13.2 torch-2.6.0 MPS (Apple M4 Pro)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/Users/ihanjo/Documents/K-Digital/deepAI_Pytorch_my/4-Yolov8/YOLOv8/Aquarium_DataSet/Aquarium_Data.yaml, epochs=50, time=None, patience=30, batch=64, imgsz=416, save=True, save_period=-1, cache=False, device=mps, workers=8, project=/Users/ihanjo/Documents/K-Digital/deepAI_Pytorch_my/4-Yolov8/YOLOv8, name=modelresult, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=Non

train: Scanning /Users/ihanjo/Documents/K-Digital/deepAI_Pytorch_my/4-Yolov8/YOLOv8/Aquarium_DataSet/train/labels... 448 images, 1 backgrounds, 0 corrupt: 100%|██████████| 448/448 [00:00<00:00, 3789.03it/s]

train: New cache created: /Users/ihanjo/Documents/K-Digital/deepAI_Pytorch_my/4-Yolov8/YOLOv8/Aquarium_DataSet/train/labels.cache



val: Scanning /Users/ihanjo/Documents/K-Digital/deepAI_Pytorch_my/4-Yolov8/YOLOv8/Aquarium_DataSet/valid/labels... 127 images, 0 backgrounds, 0 corrupt: 100%|██████████| 127/127 [00:00<00:00, 4966.17it/s]

...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:24<00:00, 24.77s/it]

                   all        127        909      0.565      0.113      0.126     0.0625
EarlyStopping: Training stopped early as no improvement observed in last 30 epochs. Best results observed at epoch 9, best model saved as best.pt.
To update EarlyStopping(patience=30) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



39 epochs completed in 0.311 hours.
Optimizer stripped from /Users/ihanjo/Documents/K-Digital/deepAI_Pytorch_my/4-Yolov8/YOLOv8/modelresult/weights/last.pt, 6.2MB
Optimizer stripped from /Users/ihanjo/Documents/K-Digital/deepAI_Pytorch_my/4-Yolov8/YOLOv8/modelresult/weights/best.pt, 6.2MB

Validating /Users/ihanjo/Documents/K-Digital/deepAI_Pytorch_my/4-Yolov8/YOLOv8/modelresult/weights/best.pt...
Ultralytics 8.3.95 🚀 Python-3.13.2 torch-2.6.0 MPS (Apple M4 Pro)
Model summary (fused): 72 layers, 3,007,013 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:10<00:00, 10.99s/it]


                   all        127        909      0.604     0.0963      0.358      0.191
                  fish         63        459      0.827     0.0828      0.481      0.233
             jellyfish          9        155          1     0.0193      0.832       0.44
               penguin         17        104      0.518     0.0865      0.148     0.0599
                puffin         15         74      0.114    0.00308      0.107     0.0424
                 shark         28         57      0.256      0.316       0.26      0.132
              starfish         17         27          1      0.106      0.328      0.212
              stingray         23         33      0.517     0.0606       0.35      0.219
Speed: 0.6ms preprocess, 12.5ms inference, 0.0ms loss, 50.3ms postprocess per image
Results saved to /Users/ihanjo/Documents/K-Digital/deepAI_Pytorch_my/4-Yolov8/YOLOv8/modelresult
모델 학습 완료. 저장 경로: /Users/ihanjo/Documents/K-Digital/deepAI_Pytorch_my/4-Yolov8/YOLOv8/modelresult


In [16]:
# YOLOv8 모델로 예측 실행 및 저장 경로 변경
# 사용자 정의 YOLOv8 모델 로드
model = YOLO("/Users/ihanjo/Documents/K-Digital/deepAI_Pytorch_my/4-Yolov8/YOLOv8/modelresult/weights/best.pt")

results = model.predict(
    source='/Users/ihanjo/Documents/K-Digital/deepAI_Pytorch_my/4-Yolov8/YOLOv8/IMG_2301_jpeg_jpg.rf.2c19ae5efbd1f8611b5578125f001695.jpg',  # 테스트 이미지 경로
    save=True,  # 결과 저장 활성화
    save_dir='/Users/ihanjo/Documents/K-Digital/deepAI_Pytorch_my/4-Yolov8/YOLOv8/modelresultimg/'  # 결과 저장 경로
)

print(f"결과 이미지가 다음 경로에 저장되었습니다: /Users/ihanjo/Documents/K-Digital/deepAI_Pytorch_my/4-Yolov8/YOLOv8/modelresultimg/")


image 1/1 /Users/ihanjo/Documents/K-Digital/deepAI_Pytorch_my/4-Yolov8/YOLOv8/IMG_2301_jpeg_jpg.rf.2c19ae5efbd1f8611b5578125f001695.jpg: 416x320 (no detections), 13.1ms
Speed: 0.7ms preprocess, 13.1ms inference, 0.3ms postprocess per image at shape (1, 3, 416, 320)
Results saved to runs/detect/predict
결과 이미지가 다음 경로에 저장되었습니다: /Users/ihanjo/Documents/K-Digital/deepAI_Pytorch_my/4-Yolov8/YOLOv8/modelresultimg/


In [ ]:
# 비디오 예측 실행
video_path = "/Users/your_username/Documents/YOLOTest7/sample_video.mp4"
output_video = os.path.join(base_path, "output_video.mp4")

cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("Error: Cannot open video file.")
    exit()

fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*"mp4v")

out = cv2.VideoWriter(output_video, fourcc, fps, (width, height))

frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model.predict(source=frame, save=False, conf=0.5)
    annotated_frame = results[0].plot()
    out.write(annotated_frame)

cap.release()
out.release()
print(f"Detection completed. Output video saved at {output_video}")
